In [18]:
import pandas as pd
import torch
from torch import nn, optim
import numpy as np
from torch.utils.tensorboard import SummaryWriter


In [12]:
data_df= pd.read_csv(r'D:\python\PYTORCH\blender_models\ik_blender\ik_data.csv',index_col=False)
location_lable = data_df.iloc[:,1:3]
angle_input = data_df.iloc[:,3:]

input_array = location_lable.values
lable_array = angle_input.values

device = torch.device('cuda:0')

#just converting it to tensor
inputs_tensor = torch.tensor(input_array,dtype=torch.float32).to(device)
labels_tensor = torch.tensor(lable_array,dtype=torch.float32).to(device)

dataset = torch.utils.data.TensorDataset(inputs_tensor, labels_tensor)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=420, shuffle=True)

In [29]:
writer = SummaryWriter(r'D:\python\PYTORCH\blender_models\log\v3\ik_problemv5')

In [30]:
IkPrdt = nn.Sequential(nn.Linear(2,20),
                       nn.ReLU(),
                       nn.Linear(20,40),
                       nn.ReLU(),
                       nn.Linear(40,80),
                       nn.ReLU(),
                       nn.Linear(80,400),
                       nn.ReLU(),
                       nn.Linear(400,60),
                       nn.ReLU(),
                       nn.Linear(60,30),
                       nn.ReLU(),
                       nn.Linear(30,15),
                       nn.ReLU(),
                       nn.Linear(15,3)
                      )

IkPrdt.to(device)
criterion = nn.HuberLoss()
optimizer = optim.SGD(IkPrdt.parameters(), lr=0.01)
epoch = 100

In [31]:
i = 0
while True:
    running_loss = 0
    for inputs_tensor, labels_tensor in train_loader:
        optimizer.zero_grad()

        output = IkPrdt(inputs_tensor)
        loss  = criterion(output, labels_tensor)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

            # every 1000 mini-batches...

            # ...log the running loss
        writer.add_scalar('training loss',running_loss, i * len(train_loader) + i)
        i += 1

    print(running_loss)

1467.3585968017578
1467.765365600586
1466.9361114501953
1465.1239624023438
1467.2519989013672
1466.0062255859375
1466.4288330078125
1467.1754913330078
1466.5702362060547
1464.8011932373047
1464.9503479003906
1465.2991790771484
1462.9403839111328
1461.7764129638672
1463.2533569335938
1461.1963500976562
1459.7490234375
1459.5272369384766
1461.4610900878906
1454.9107666015625
1455.2081451416016
1456.8636322021484
1456.1395111083984
1453.4963836669922
1449.9384460449219
1450.3338012695312
1443.2032470703125
1434.5965118408203
1403.8496551513672
1148.4407501220703
1342.7358894348145
1441.2850341796875
1436.1837921142578
1427.5259704589844
1415.508773803711
1384.7397766113281
1246.2586975097656
568.7913475036621
559.1376533508301
574.5093040466309
450.00146484375
474.5729217529297
450.19898223876953
434.5530548095703
410.0255317687988
469.80624771118164
431.187068939209
399.9329833984375
511.1882438659668
437.65603256225586
423.1773452758789
494.13396072387695
400.9474639892578
503.138954162

KeyboardInterrupt: 

In [32]:
torch.save(IkPrdt.state_dict(),"./v6.pth")